# GeoJSON

A standard for representing geographic features in JavaScript Object Notation (JSON).


For more background on this section see this blog post:
https://dzone.com/articles/enriching-geojson-data-to-render-a-map-of-smart-ci

In [ ]:
import json
import base64
import requests

def get_client_token():
    uri = 'https://auth.aa.cityiq.io/oauth/token'
    credentials = base64.b64encode(b'PublicAccess:uVeeMuiue4k=').decode('ascii')
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Cache-Control': 'no-cache',
        'Authorization': 'Basic ' + credentials
    }
    
    params = {
        'grant_type': 'client_credentials'
    }
    response = requests.post(uri, headers=headers, params=params)
    return json.loads(response.text)['access_token']

In [ ]:
token = get_client_token()
token

In [ ]:
def get_assets(token):
    uri = 'https://sandiego.cityiq.io/api/v2/metadata/assets/search'
    headers = {
       'Content-Type': 'application/x-www-form-urlencoded',
       'Predix-Zone-Id': 'SD-IE-TRAFFIC',
       'Authorization': 'Bearer ' + token
       }
    # Bounding Box for San Diego Camera locations because these are the
    # nodes that have traffic, parking and pedestrian data
    params = {
            'bbox': '33.077762:-117.663817,32.559574:-116.584410',
            'page': 0,
            'size': 200,
            'q': 'assetType:CAMERA'
            }
    response = requests.get(uri, headers=headers, params=params)
    return response.json()

In [ ]:
token = get_client_token()
assets = get_assets(token)
assets

In [ ]:
import geojson

def get_features(assets):
    collection = []
    for asset in assets['content']:
        coords = asset['coordinates']
        lat,lon = coords.split(':')
        
        point = geojson.Point(coordinates=(float(lon), float(lat)))
        feature = geojson.Feature(geometry=point, properties=asset)
        collection.append(feature)
        
    return collection

In [ ]:
collection = get_features(assets)
data = geojson.FeatureCollection(collection)
data

In [ ]:
with open('city.geojson', 'w') as output:
    geojson.dump(data, output)

# Try It

With a GeoJSON data file like this there are a number of online viewers.  Try to create GeoJSON of some features relevant to you and upload to:

- https://geojson.tools
- https://here.xyz

